In [1]:
import textworld
from textworld import GameMaker
from textworld.generator.data import KnowledgeBase
from textworld.generator.game import Event, Quest
from textworld.generator.game import GameOptions

# Make the generation process reproducible.
from textworld import g_rng  # Global random generator.
g_rng.set_seed(20180916)

In [2]:
from textworld.generator import compile_game
import io
import sys
import contextlib
import numpy as np

In [3]:
# GameMaker object for handcrafting text-based games.
# M = GameMaker()

# Defining the layout of the world

In [4]:
def get_game_maker():
    M = GameMaker()
    roomA = M.new_room("Room A")
    alley = M.new_room("Alley")
    bank1 = M.new_room("Bank1")
    bank2 = M.new_room("Bank2")
    bank3 = M.new_room("Bank3")
    corridor = M.connect(roomA.east, alley.west)
    corridor1 = M.connect(alley.east, bank1.west)
    corridor1 = M.connect(alley.north, bank2.south)
    corridor1 = M.connect(alley.south, bank3.north)
    M.set_player(roomA)
    
    roomA.infos.desc = "You are in a road. Some mobs are planning to rob a bank. You need to stop them. Go east to the alley. You can find a person in the alley who has information about the roberry. Collect information from him and prevent the roberry."
    alley.infos.desc = "There is a person beside the table in the alley. You can find an oven here as well."
    supporter = M.new(type='s', name = "table")  # When not provided, names are automatically generated.
    alley.add(supporter)  # Supporters are fixed in place.
    supporter.infos.desc = "It is a metal sturdy table. There are some food on the table"
    food = M.new(type="f", name = 'carrot') 
    food.infos.desc = "It's carrot"
    stove = M.new(type="oven", name = "oven")
    stove.infos.desc = "this is an oven. you can cook your food"
    supporter.add(food)  # When added directly to a room, portable objects are put on the floor.
    #supporter.add(stove)
    alley.add(stove)
    person = M.new(type="pr", name = "informant")
    person.infos.desc = "This person knows about the bank roberry. Do a favor for him. He will help you."
    M.add_fact("not_asked", person)
    alley.add(person)
    M.add_fact("raw",food)
    robber = M.new(type="rbr", name = "joker")
    bank2.add(robber)
    M.add_fact("not_stopped", robber)
    M.render()
    return M

In [5]:
def get_game_maker_1():
    M = GameMaker()
    roomA = M.new_room("Room A")
    alley = M.new_room("Alley")
    bank1 = M.new_room("Bank1")
    bank2 = M.new_room("Bank2")
    bank3 = M.new_room("Bank3")
#     police_station = M.new_room("Police Station")
    corridor = M.connect(roomA.east, alley.west)
    corridor1 = M.connect(alley.east, bank1.west)
    corridor2 = M.connect(alley.north, bank2.south)
    corridor3 = M.connect(alley.south, bank3.north)
    
#     corridor4 = M.connect(roomA.west, police_station.east)
    #door = M.new_door(corridor4, name="door")  # Door and room have their own "new" method, no need to provide the type.
    #M.add_fact("locked", door)
    
    M.set_player(roomA)
    
    roomA.infos.desc = "You are in a road. Some mobs are planning to rob a bank. You need to stop them. Go east to the alley. You can find a person in the alley who has information about the roberry. Collect information from him and prevent the roberry."
    alley.infos.desc = "This is the alley. There is a person beside the table. You can find an oven here as well."
    
    money = M.new(type="o", name = 'money') 
    money.infos.desc = "it is money"
    M.inventory.add(money) 
    person = M.new(type="pr", name = "informant")
    person.infos.desc = "This person knows about the bank roberry. Do a favor for him. He will help you."
    M.add_fact("not_asked", person)
    M.add_fact("not_given", person)
    alley.add(person)
    
    robber = M.new(type="rbr", name = "joker")
    bank2.add(robber)
    M.add_fact("not_stopped", robber)
    M.add_fact("not_allowed", robber)
    police = M.new(type="pl", name = "police")
    bank2.add(police)
    M.add_fact("not_called", police)
#     M.add_fact("not_called", robber)
    M.render()
    return M

In [6]:
# M = get_game_maker_1()
# quest = M.record_quest()

In [7]:
@contextlib.contextmanager
def capture_stdout():
    # Capture stdout.
    stdout_bak = sys.stdout
    sys.stdout = out = io.StringIO()
    try:
        yield out
    finally:
        # Restore stdout
        sys.stdout = stdout_bak


def _compile_test_game(game):
    grammar_flags = {
        "theme": "house",
        "include_adj": False,
        "only_last_action": True,
        "blend_instructions": True,
        "blend_descriptions": True,
        "refer_by_name_only": True,
        "instruction_extension": []
    }
    rng_grammar = np.random.RandomState(1234)
    grammar = textworld.generator.make_grammar(grammar_flags, rng=rng_grammar)
    game.change_grammar(grammar)

    game_file = textworld.generator.compile_game(game)
    return game_file


def build_and_compile_no_quest_game(options: GameOptions):
    M = textworld.GameMaker()

    room = M.new_room()
    M.set_player(room)
    item = M.new(type="o")
    room.add(item)
    game = M.build()

    game_file = _compile_test_game(game, options)
    return game, game_file

def build_and_compile_super_hero_game_1():
    M = GameMaker()
    roomA = M.new_room("Room A")
    alley = M.new_room("Alley")
    bank1 = M.new_room("Bank1")
    bank2 = M.new_room("Bank2")
    bank3 = M.new_room("Bank3")
    corridor = M.connect(roomA.east, alley.west)
    corridor1 = M.connect(alley.east, bank1.west)
    corridor1 = M.connect(alley.north, bank2.south)
    corridor1 = M.connect(alley.south, bank3.north)
    M.set_player(roomA)
    
    roomA.infos.desc = "You are in a road. Some mobs are planning to rob a bank. You need to stop them. Go east to the alley. You can find a person in the alley who has information about the roberry. Collect information from him and prevent the roberry."
    alley.infos.desc = "This is an alley. There is a person beside the table. He knows about the bank roberry."
    bank2.infos.desc = "This is the north bank. Some robbers are going to rob the bank. You can call the police and try to capture them or convince them to surrender. Or you can also shoot them to stop the robbery."
    
    money = M.new(type="o", name = 'money') 
    money.infos.desc = "it is money"
    M.inventory.add(money) 
    person = M.new(type="pr", name = "informant")
    person.infos.desc = "This person knows about the bank roberry. Do a favor for him. He will help you."
    M.add_fact("not_asked", person)
    M.add_fact("not_given", person)
    alley.add(person)
    
    robber = M.new(type="rbr", name = "joker")
    bank2.add(robber)
    M.add_fact("not_stopped", robber)
    
#     M.add_fact("not_conflict", robber)
    M.add_fact("not_allowed", robber)
    
    police = M.new(type="pl", name = "police")
    bank2.add(police)
    M.add_fact("not_called", police)
    
    # cooking food quest
#     quest1_cmds = ["go east", "take the carrot", "cook the carrot"]
#     failed_cmds1 = quest1_cmds + ["go south"]
#     qst_event_cook_carrot = M.new_event_using_commands(quest1_cmds)
#     failed_event_1 = M.new_event_using_commands(failed_cmds1)
    
#     quest1 = Quest(win_events=[qst_event_cook_carrot],
#                    fail_events=[failed_event_1],
#                    reward=2)
    
    #quest1 = Quest(win_events=[qst_event_cook_carrot],
    #               reward=2)
    
    # asking quest
    qst_event_asking = Event(conditions={M.new_fact("asked", person)})
    quest_asking = Quest(win_events=[qst_event_asking],
                      reward=2)
    # the wining quest
    qst_event_stopped_rob = Event(conditions={M.new_fact("asked", person),
                                             M.new_fact("stopped", robber)})
    win_quest = Quest(win_events=[qst_event_stopped_rob],
                      reward=2)

    # 1st failure condition
    failed_cmds1 = ["go east", "go south"]
    failed_event1 = M.new_event_using_commands(failed_cmds1)
    failed_quest_1 = Quest(win_events=[],
                           fail_events=[failed_event1])
    
    # 2nd failure condition
    failed_cmds2 = ["go east", "go east"]
    failed_event2 = M.new_event_using_commands(failed_cmds2)
    failed_quest_2 = Quest(win_events=[],
                           fail_events=[failed_event2])
    
    # 3rd failure condition
    failed_event3 = Event(conditions={
        M.new_fact("not_asked", person),
        M.new_fact("at", M._entities['P'], bank2)})
    
#     failed_cmds3 = ["go east", "go north"]
#     failed_actions = M.new_event_using_commands(failed_cmds3).actions
#     failed_event3 = Event(conditions={M.new_fact("not_asked", person)},
#                          actions=failed_actions)
    failed_quest_3 = Quest(win_events=[],
                           fail_events=[failed_event3])
    
    
    failed_event4 = Event(conditions={
        M.new_fact("allowed", robber)})
    
    failed_quest_4 = Quest(win_events=[],
                   fail_events=[failed_event4])
    

    #     quest2 = Quest(win_events=[ask_the_informant],
#                    fail_events=[eating_carrot])
#     ask_the_informant = Event(conditions={M.new_fact("asked", person)},
#                               actions=quest2_actions)
#     quest2 = Quest(win_events=[ask_the_informant],
#                    fail_events=[eating_carrot])

    
#     failed_event4 = Event(conditions={
#         M.new_fact("burned", food)})
#     failed_quest_4 = Quest(win_events=[],
#                            fail_events=[failed_event4])
    
    
    
#     failed_event_2 = M.new_event_using_commands(failed_cmds2)
#     eating_carrot = Event(conditions={M.new_fact("consumed", food)})
    
#     quest2_cmds =  quest1_cmds + ["ask the informant"]
#     quest2_actions = M.new_event_using_commands(quest2_cmds).actions
#     ask_the_informant = Event(conditions={M.new_fact("asked", person)},
#                               actions=quest2_actions)
#     quest2 = Quest(win_events=[ask_the_informant],
#                    fail_events=[eating_carrot])
    
#     ask_the_informant = M.new_event_using_commands(quest2_cmds)
#     quest2 = Quest(win_events=[ask_the_informant],
#                    fail_events=[eating_carrot],
#                    reward=2)

#     quest3 = Quest(win_events=[],
#                    fail_events=[failed_event_2])
    
#     M.quests = [quest1, win_quest, failed_quest_1, failed_quest_2, failed_quest_3]
#     M.quests = [quest1, win_quest, quest_asking, failed_quest_1, failed_quest_2, failed_quest_3]
    M.quests = [win_quest, quest_asking, failed_quest_1, failed_quest_2, failed_quest_3, failed_quest_4]
    game = M.build()
#     game.main_quest = M.new_quest_using_commands(quest2_cmds)
    game.main_quest = win_quest
    game_file = _compile_test_game(game)
    return game, game_file

def build_and_compile_super_hero_game():
    quest1_cmds = ["go east", "take carrot", "cook the carrot"]
    cook_carrot = M.new_event_using_commands(quest1_cmds)
    eating_carrot = Event(conditions={M.new_fact("consumed", food)})
    quest1 = Quest(win_events=[cook_carrot],
                   fail_events=[eating_carrot],
                   reward=2)
    
#     quest2_cmds =  ["go east", "ask the informant"]
    quest2_cmds =  quest1_cmds + ["ask the informant"]
    quest2_actions = M.new_event_using_commands(quest2_cmds).actions
    ask_the_informant = Event(conditions={M.new_fact("asked", person)},
                              actions=quest2_actions)
    quest2 = Quest(win_events=[ask_the_informant],
                   fail_events=[eating_carrot])
    
#     ask_the_informant = M.new_event_using_commands(quest2_cmds)
#     quest2 = Quest(win_events=[ask_the_informant],
#                    fail_events=[eating_carrot],
#                    reward=2)

    M.quests = [quest1, quest2]
    game = M.build()
#     game.main_quest = M.new_quest_using_commands(quest2_cmds)
    game.main_quest = quest2
    game_file = _compile_test_game(game)
    return game, game_file
    

In [8]:
game, game_file = build_and_compile_super_hero_game_1()

/media/nahian/Media/CORGI_Projects/Learning_from_story/LNFStories_gitrepo/Learning_from_stories_playground/textworld/generator/inform7/world2inform7.py:91: TextworldInform7Warning: Undefined Inform7's predicate: free(r, r)
  warnings.warn(msg, TextworldInform7Warning)


In [1]:
import textworld
import textworld.gym
import gym
request_infos = textworld.EnvInfos(description=True, inventory=True, admissible_commands=True,won=True, lost=True)

# Requesting additional information should be done when registering the game.
env_id = textworld.gym.register_game('tw_games/super_hero_9.ulx', request_infos)
env = gym.make(env_id)

obs, infos = env.reset()

In [2]:
try:
    done = False
    obs, _ = env.reset()
    print(obs)
    nb_moves = 0
    while not done:
        command = input("> ")
        obs, score, done, infos = env.step(command)
        print("infos: ",infos)
        print(obs)
        nb_moves += 1

except KeyboardInterrupt:
    pass  # Press the stop button in the toolbar to quit the game.

print("Played {} steps, scoring {} points.".format(nb_moves, score))




                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
                      | $$   | $$__     \$$\/  $$   | $$          
                      | $$   | $$  \     >$$  $$    | $$          
                      | $$   | $$$$$    /  $$$$\    | $$          
                      | $$   | $$_____ |  $$ \$$\   | $$          
                      | $$   | $$     \| $$  | $$   | $$          
                       \$$    \$$$$$$$$ \$$   \$$    \$$          
              __       __   ______   _______   __        _______  
             |  \  _  |  \ /      \ |       \ |  \      |       \ 
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$  | $$
             | $$ $$\$$\$$| $$  | $$| $$$$$$$\| $$      | $

In [12]:
import torch

In [37]:
weights = torch.tensor([0, 10, 10, 90, 20, 30, 40, 11, 15], dtype=torch.float) # create a tensor of weights


In [38]:
torch.multinomial(weights, 1)

tensor([5])

In [27]:
np.random.random()

0.11438585021230763

In [44]:
total_number = 0
for i in range(1000):
    rand_no = np.random.random()
    if rand_no <= 0.3:
        total_number += 1

In [45]:
total_number

294

In [60]:
action_index = weights.multinomial(num_samples=1).unsqueeze(0)

In [61]:
action_index

tensor([[3]])

In [56]:
action_index = weights.max(0).indices.unsqueeze(-1).unsqueeze(-1)

In [57]:
action_index

tensor([[3]])

In [75]:
weights = torch.tensor([0.0988, 0.0988, 0.0988, 0.0993, 0.0988, 0.1004, 0.0989, 0.1062, 0.0988, 0.1015],dtype=torch.float)


In [76]:
torch.multinomial(weights, 1)

tensor([6])

In [77]:
probs = torch.tensor([[[0.0988, 0.0988, 0.0988, 0.0993, 0.0988, 0.1004, 0.0989, 0.1062, 0.0988, 0.1015]]], dtype=torch.float)

In [78]:
tp = np.random.choice(probs[0][0].detach().numpy())

In [79]:
tp

0.0988

In [80]:
action_index = (probs[0][0] == tp).nonzero().unsqueeze(-1)

In [86]:
_action_index = action_index[0]

In [89]:
(probs[0][0] == tp).nonzero()

tensor([[0],
        [1],
        [2],
        [4],
        [8]])

In [85]:
probs[0][0].shape

torch.Size([10])

In [88]:
_action_index.shape

torch.Size([1, 1])

In [106]:
torch.randint(0, probs[0][0].shape[0], (1,)).unsqueeze(-1).unsqueeze(-1)

tensor([[[4]]])

In [99]:
probs[0][0].shape[0]

10